In [213]:
from tensorflow import keras 
import tensorflow  as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
print(tf.__version__)

2.1.0


### تعبیه سازی کلمات

In [ ]:
tf.keras.layers.Embedding(input_dim = 1000,
                         output_dim= 32,
                         input_length= max_length)

ابعاد ماتریس تعبیه سازی ابرپارامترهایی هستند که می توانیم آن ها را تغییر بدهیم و با سعی و خطا آن ها را انتخاب کنیم.
وقتی یک ماتریس تعبیه سازی می سازیم مولفه های این ماتریس به صورت تصادفی توسط کراس مقداردهی می شوند و ما سعی داریم در فرآیند آموزش شبکه عصبی پارامترهای این ماتریس تعبیه سازی را یاد بگیریم (بهینه سازی کنیم).

In [ ]:
لایه های تعبیه سازی تعداد زیادی پارامتر به مدل اضافه می کنند.

### یک مثال

In [107]:
spotify = pd.read_csv('data/spotify_songs_lyrics_lang.csv')
spotify = spotify[spotify.language == 'en']
spotify[['playlist_genre','full_lyrics']].head()                    

,playlist_genre,full_lyrics
1,rock,"The trees, are singing in the wind The sky blu..."
2,r&b,"NA Yeah, Spyderman and Freeze in full effect U..."
3,r&b,I really can't stay Baby it's cold outside I'v...
4,pop,Get up out of my business You don't keep me fr...
5,r&b,"Hold your breath, don't look down, keep trying..."


In [130]:
sentences = spotify['full_lyrics']
labels = spotify.playlist_genre

In [155]:
spotify.playlist_genre.unstackique()

array(['rock', 'r&b', 'pop', 'edm', 'rap', 'latin'], dtype=object)

In [141]:
labels_int = labels.astype('category').cat.codes

In [142]:
from sklearn.model_selection import train_test_split
train_sentences,test_sentences,train_labels,test_labels = train_test_split(sentences,
                                                                          labels_int,
                                                                           train_size = .8,
                                                                          random_state = 42)


In [143]:
train_sentences.shape

(12324,)

In [144]:
spotify['max_len'] = spotify['full_lyrics'].apply(lambda x:len(x.split(' ')))
spotify['max_len'].median()

332.0

In [166]:
vocab_size = 10000
num_classes = 6
embedding_dim = 32
max_length = 350
trunc_type='post'
oov_tok = "<OOV>"
num_epochs = 20
batch_size = 8

In [146]:
tokenizer = Tokenizer(num_words = vocab_size,
                     oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

In [147]:
index_word = tokenizer.index_word
#word_index
#word_index

### Padding 

In [ ]:
index_word = tokenizer.index_word
for sequence in sequences:
    print(len(sequence))

In [148]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences,maxlen=max_length, truncating=trunc_type)

In [149]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences,maxlen=max_length, truncating=trunc_type)

In [152]:
len(train_sentences)

12324

In [154]:
train_sentences.iloc[0],train_sequences[0]

('We talk to the deep of the night Moving by cigarette light Waiting for something to spark But you left me in the dark Mistakes that you made in the past Cutting through you like they\'re glass There are no words that can paint The longing in your face I just want to hold you Keep you in my world But you wanted to go higher Take a chance and explode Thought I was more than the afterlife Thought I was more than the You\'re at the edge of your desire, left the story untold Thought I was more than the afterlife Thought I was more than the, more than the More than, baby Ohh ohh ohh ohhh Love\'s never clear in a haze But you feel good in a sway Calling me up for a ride I pick you up, we fight I say "the future is yours if you choose And mine won\'t exist without you" Nothing I say hits your heart You keep me in the dark I just want to hold you Keep you in my world But you wanted to go higher Take a chance and explode Thought I was more than the afterlife Thought I was more than the You\'re

In [160]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 10000,
                             output_dim = 32,
                             input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'softmax')
    
])

In [161]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 350, 32)           320000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 11200)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               1120100   
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 606       
Total params: 1,440,706
Trainable params: 1,440,706
Non-trainable params: 0
_________________________________________________________________


In [168]:
e = model.layers[0]
emb_weights = e.get_weights()[0]
print(emb_weights)

[[-0.00031968  0.02499335  0.00698107 ... -0.01133183  0.00913798
   0.00727829]
 [-0.01444615  0.00676631  0.02981719 ...  0.06844001 -0.09848961
   0.0450506 ]
 [-0.03000842  0.02605446 -0.03474898 ...  0.03370769  0.01117229
   0.05936031]
 ...
 [ 0.03575659  0.04730195 -0.04181882 ... -0.03387503 -0.02601899
   0.0409003 ]
 [-0.2755366  -0.03717608  0.06744698 ... -0.01152058  0.11854044
   0.00436996]
 [ 0.03523178  0.06736675 -0.01424573 ... -0.02959427  0.00559779
   0.0828836 ]]


In [162]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [164]:
train_padded.shape,train_labels.shape

((12324, 350), (12324,))

In [167]:
model.fit(train_padded,
          train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          validation_data=(test_padded,test_labels)
         )

Train on 12324 samples, validate on 3081 samples
Epoch 1/20
12324/12324 [==============================] - 18s 1ms/sample - loss: 1.3916 - accuracy: 0.4686 - val_loss: 1.2624 - val_accuracy: 0.5362
Epoch 2/20
12324/12324 [==============================] - 16s 1ms/sample - loss: 0.6864 - accuracy: 0.7714 - val_loss: 1.5533 - val_accuracy: 0.4830
Epoch 3/20
12324/12324 [==============================] - 17s 1ms/sample - loss: 0.2452 - accuracy: 0.9258 - val_loss: 1.8536 - val_accuracy: 0.4726
Epoch 4/20
12324/12324 [==============================] - 18s 1ms/sample - loss: 0.1584 - accuracy: 0.9415 - val_loss: 2.0372 - val_accuracy: 0.4872
Epoch 5/20
12324/12324 [==============================] - 17s 1ms/sample - loss: 0.1258 - accuracy: 0.9417 - val_loss: 2.2069 - val_accuracy: 0.4849
Epoch 6/20
12324/12324 [==============================] - 17s 1ms/sample - loss: 0.1227 - accuracy: 0.9447 - val_loss: 2.3348 - val_accuracy: 0.4920
Epoch 7/20
12324/12324 [==============================] -

In [170]:
preds = model.predict_classes(test_padded)

In [174]:
pred = pd.DataFrame({'pred':preds,
                     'sentence':test_sentences,
                    'genre':test_labels})

In [178]:
pred.head(10)

,pred,sentence,genre
3556,3,Caroline laughs and it's raining all day She l...,5
8209,2,"Where the hoes, where the hoes Lookin', lookin...",4
8038,5,"Yea, we in the clear Put it in drive, we outta...",0
4159,3,"It's Kid Ink, yeah yeah, watch this It's not a...",2
922,2,"My nigga, C, he from Texas, I'm from Tennesee ...",4
16669,2,"Thirty days, lord and thirty nights I'm coming...",5
2134,0,How can I see what I'm needing for How can I g...,5
12722,3,"Oh girl, I see you, the time is so right Let's...",3
17055,3,Seems I've got to have a change of scene Every...,5
18137,1,"Boy, you make it sound so pretty even when it'...",0


In [177]:
test_padded.shape

(3081, 350)

In [ ]:
e = model.layers[0]
emb_weights = e.get_weights()[0]
print(emb_weights)

### شبکه های کانولوشنال

In [180]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 10000,
                             output_dim = 32,
                             input_length = max_length),
    tf.keras.layers.Conv1D(128,5,activation = 'relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'softmax')
    
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(train_padded,
          train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          validation_data=(test_padded,test_labels)
         )

Train on 12324 samples, validate on 3081 samples
Epoch 1/20
12324/12324 [==============================] - 17s 1ms/sample - loss: 1.4813 - accuracy: 0.4068 - val_loss: 1.3615 - val_accuracy: 0.4791
Epoch 2/20
12324/12324 [==============================] - 15s 1ms/sample - loss: 1.2607 - accuracy: 0.5272 - val_loss: 1.2440 - val_accuracy: 0.5472
Epoch 3/20
12324/12324 [==============================] - 14s 1ms/sample - loss: 1.1046 - accuracy: 0.5983 - val_loss: 1.2368 - val_accuracy: 0.5437
Epoch 4/20
12324/12324 [==============================] - 16s 1ms/sample - loss: 0.9771 - accuracy: 0.6467 - val_loss: 1.2708 - val_accuracy: 0.5407
Epoch 5/20
12324/12324 [==============================] - 16s 1ms/sample - loss: 0.8644 - accuracy: 0.6919 - val_loss: 1.3140 - val_accuracy: 0.5440
Epoch 6/20
12324/12324 [==============================] - 16s 1ms/sample - loss: 0.7602 - accuracy: 0.7278 - val_loss: 1.4175 - val_accuracy: 0.5368
Epoch 7/20
12324/12324 [==============================] -

### LSTM

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 10000,
                             output_dim = 32,
                             input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'softmax')
    
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(train_padded,
          train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          validation_data=(test_padded,test_labels)
         )

Train on 12324 samples, validate on 3081 samples
Epoch 1/20
12324/12324 [==============================] - 249s 20ms/sample - loss: 1.5390 - accuracy: 0.3682 - val_loss: 1.4344 - val_accuracy: 0.4297
Epoch 2/20
12324/12324 [==============================] - 205s 17ms/sample - loss: 1.4006 - accuracy: 0.4454 - val_loss: 1.4363 - val_accuracy: 0.4323
Epoch 3/20
12324/12324 [==============================] - 222s 18ms/sample - loss: 1.2061 - accuracy: 0.5467 - val_loss: 1.4127 - val_accuracy: 0.4518
Epoch 4/20
12324/12324 [==============================] - 212s 17ms/sample - loss: 1.0988 - accuracy: 0.5935 - val_loss: 1.4424 - val_accuracy: 0.4609
Epoch 5/20
12324/12324 [==============================] - 241s 20ms/sample - loss: 0.9566 - accuracy: 0.6474 - val_loss: 1.5150 - val_accuracy: 0.4648
Epoch 6/20
12324/12324 [==============================] - 256s 21ms/sample - loss: 0.8540 - accuracy: 0.6951 - val_loss: 1.6298 - val_accuracy: 0.4512
Epoch 7/20
12324/12324 [=====================

In [186]:
vocab_size = 10000
num_classes = 6
embedding_dim = 64
max_length = 700
trunc_type='post'
oov_tok = "<OOV>"
num_epochs = 20
batch_size = 8


tokenizer = Tokenizer(num_words = vocab_size,
                     oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences,maxlen=max_length, truncating=trunc_type)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences,maxlen=max_length, truncating=trunc_type)




In [190]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 10000,
                             output_dim = embedding_dim,
                             input_length = max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'softmax')
    
])


In [191]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(train_padded,
          train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          validation_data=(test_padded,test_labels)
         )

Train on 12324 samples, validate on 3081 samples
Epoch 1/20
12324/12324 [==============================] - 252s 20ms/sample - loss: 1.5340 - accuracy: 0.3681 - val_loss: 1.4407 - val_accuracy: 0.4288
Epoch 2/20
12324/12324 [==============================] - 194s 16ms/sample - loss: 1.3299 - accuracy: 0.4945 - val_loss: 1.3024 - val_accuracy: 0.5115
Epoch 3/20
12324/12324 [==============================] - 207s 17ms/sample - loss: 1.1762 - accuracy: 0.5699 - val_loss: 1.2888 - val_accuracy: 0.5268
Epoch 4/20
12324/12324 [==============================] - 222s 18ms/sample - loss: 1.0552 - accuracy: 0.6222 - val_loss: 1.3751 - val_accuracy: 0.5073
Epoch 5/20
12324/12324 [==============================] - 220s 18ms/sample - loss: 0.9463 - accuracy: 0.6676 - val_loss: 1.3585 - val_accuracy: 0.5167
Epoch 6/20
12324/12324 [==============================] - 199s 16ms/sample - loss: 0.8672 - accuracy: 0.6985 - val_loss: 1.4087 - val_accuracy: 0.5063
Epoch 7/20
12324/12324 [=====================

### GRU

In [182]:
1

1

### Padding

### متن های فارسی

In [32]:
#import hazm 

In [59]:
factnameh = pd.read_csv('data/factnameh.csv')
factnameh = factnameh[factnameh.conclusion_sum.isin(['false',
                                                     'misleading',
                                                    'outrageous',
                                                    'true',
                                                    'almost_true',
                                                    'no_data'])]
factnameh.shape

(337, 72)

In [60]:
factnameh.head()

,subject,claim,tags,entity,entity_role,date,body_text,conclusion_sum,conclusion,links,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,آمار روحانی درباره نرخ تورم در دولت یازدهم,در دولت یازدهم نرخ تورم از ۴۰ درصد به ۷ درصد ر...,اقتصاد,حسن روحانی,\n رییس‌جمهور ایران...,۱۳۹۹/۳/۲۴,حسن روحانی، رئیس جمهوری ایران روز چهارشنبه ۲۱ ...,false,\n گفته یا آمار، نادرست است یا دست‌کم سندی...,http://dolat.ir/detail/340695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,نرخ رشد اقتصادی در دولت اول روحانی چقدر بوده است؟,پیش از تحریم‌های شدید، نرخ رشد اقتصادی را از م...,اقتصاد,حسن روحانی,\n رییس‌جمهور ایران...,۱۳۹۹/۳/۲۴,حسن روحانی، رئیس جمهوری ایران روز چهارشنبه ۲۱ ...,misleading,\n گفته یا آمار، نادرست نیست اما به گونه‌ای...,http://dolat.ir/detail/340695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,هیچ طرحی برای ازدواج اجباری به مجلس ارائه نشده...,ارائه طرح ازدواج اجباری به مجلس شورای اسلامی\n,NaN,شبکه‌های اجتماعی,\n تلگرام، توییتر، ...,۱۳۹۹/۳/۲۲,فضای مجازی شرح ماجرا این طرح که بیشتر با عنوا...,false,\n گفته یا آمار، نادرست است یا دست‌کم سندی...,https://www.instagram.com/explore/tags/%D8%A7%...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,حجم تخلفات درون دستگاه قضایی ایران,حجم تخلفات درون دستگاه قضایی کمتر از یک درصد است.,NaN,محمد موحدی آزاد,\n دادستان انتظامی ...,۱۳۹۹/۳/۲۲,محمد موحدی آزاد، دادستان انتظامی قضات روز دوشن...,false,\n گفته یا آمار، نادرست است یا دست‌کم سندی...,https://www.ilna.news/%D8%A8%D8%AE%D8%B4-%D8%B...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ایران جزو پیرترین کشورهای جهان نخواهد شد,با توجه به آمار، ایران در سال ۱۴۳۰ جزو ۳ کشور ...,NaN,ابوالفضل ابوترابی,\n نماینده نجف‌آباد...,۱۳۹۹/۳/۲۱,ابوالفضل ابوترابی، نماینده نجف آباد در مجلس یا...,outrageous,\n گفته یا آمار، به قدری نادرست و مضحک است ...,https://twitter.com/abotoraby/status/126845051...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
labels = factnameh.conclusion_sum.astype('category').cat.codes
pd.unique(labels)
ys = tf.keras.utils.to_categorical(labels, num_classes=6)
ys

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [61]:
#factnameh[factnameh.conclusion_sum == 'no_data']

In [79]:
vocab_siz = 1000
embedding_dim = 16
max_length = 20
trunc_type = 'post'
num_epochs = 100
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_siz,oov_token= oov_tok)
tokenizer.fit_on_texts(factnameh.subject.tolist())

In [80]:
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(factnameh.subject.tolist())
factnameh.subject.tolist()[1],sequences[1]

('نرخ رشد اقتصادی در دولت اول روحانی چقدر بوده است؟',
 [40, 18, 33, 2, 35, 98, 13, 9, 99, 4])

In [81]:
len(sequences),len(ys)

(337, 337)

In [82]:
padded = pad_sequences(sequences,maxlen= max_length,truncating=trunc_type )

In [83]:
padded[1],factnameh.subject.tolist()[1]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 40, 18, 33,  2, 35, 98, 13,
         9, 99,  4]), 'نرخ رشد اقتصادی در دولت اول روحانی چقدر بوده است؟')

In [97]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_siz,
                             embedding_dim,
                             input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(6,activation = 'softmax')
    
])

In [98]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
flatten (Flatten)            (None, 320)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               32100     
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 606       
Total params: 48,706
Trainable params: 48,706
Non-trainable params: 0
_________________________________________________________________


In [99]:
model.compile(optimizer='adam',
              #loss = 'sparse_categorical_crossentropy',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [100]:
num_epochs = 100
model.fit(padded , ys,epochs=num_epochs,batch_size=8)

Train on 337 samples
Epoch 1/100
337/337 [==============================] - 1s 2ms/sample - loss: 1.6413 - accuracy: 0.4421
Epoch 2/100
337/337 [==============================] - 2s 7ms/sample - loss: 1.5003 - accuracy: 0.4599
Epoch 3/100
337/337 [==============================] - 0s 729us/sample - loss: 1.4407 - accuracy: 0.4599
Epoch 4/100
337/337 [==============================] - 0s 759us/sample - loss: 1.3348 - accuracy: 0.4688
Epoch 5/100
337/337 [==============================] - 0s 494us/sample - loss: 1.1351 - accuracy: 0.5668
Epoch 6/100
337/337 [==============================] - 0s 308us/sample - loss: 0.8676 - accuracy: 0.7834
Epoch 7/100
337/337 [==============================] - 0s 380us/sample - loss: 0.6152 - accuracy: 0.8635
Epoch 8/100
337/337 [==============================] - 0s 364us/sample - loss: 0.4272 - accuracy: 0.9080
Epoch 9/100
337/337 [==============================] - 0s 362us/sample - loss: 0.2949 - accuracy: 0.9318
Epoch 10/100
337/337 [================

In [94]:
len(padded),len(labels)

(337, 337)

In [58]:
for i in padded:
    print(len(i))

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
2

### ساختن متن های مصنوعی


In [210]:
import hazm
import parsivar

my_tokenizer = parsivar.Tokenizer()
my_normalizer = parsivar.Normalizer()

In [198]:
#!pip install parsivar

In [194]:
poems = pd.read_csv('data/poems_with_more_info.tsv',sep ='\t')
poems.head()

,poet,century,book_title,poem_title,poem
0,رودکی,3,قصاید و قطعات,شمارهٔ ۱,گر من این دوستی تو ببرم تا لب گور بزنم نعره...
1,رودکی,3,قصاید و قطعات,شمارهٔ ۲,به حق نالم ز هجر دوست زارا سحرگاهان چو بر گ...
2,رودکی,3,قصاید و قطعات,شمارهٔ ۳,به نام نیک تو، خواجه، فریفته نشوم که نام نی...
3,رودکی,3,قصاید و قطعات,شمارهٔ ۴,دلا، تا کی همی جویی منی را؟ چه داری دوست هر...
4,رودکی,3,قصاید و قطعات,شمارهٔ ۵,گرفت خواهم زلفین عنبرین ترا به بوسه نقش‌کنم...


In [199]:
poems_text = ' '.join(poems.poem)

In [201]:
len(poems_text),type(poems_text)

(46641993, str)

In [203]:
#poems_text_norm = my_normalizer.normalize(poems_text)

In [207]:
hafez_poems = poems[poems['poet']=='حافظ']
hafez_poems['']
hafez_poems.head()

,poet,century,book_title,poem_title,poem
24358,حافظ,8,رباعیات,رباعی شمارهٔ ۱,جز نقش تو در نظر نیامد ما را جز کوی تو رهگذ...
24359,حافظ,8,رباعیات,رباعی شمارهٔ ۲,بر گیر شراب طرب‌انگیز و بیا پنهان ز رقیب سف...
24360,حافظ,8,رباعیات,رباعی شمارهٔ ۳,گفتم که لبت، گفت لبم آب حیات گفتم دهنت، گفت...
24361,حافظ,8,رباعیات,رباعی شمارهٔ ۴,ماهی که قدش به سرو می‌ماند راست آیینه به دس...
24362,حافظ,8,رباعیات,رباعی شمارهٔ ۵,من باکمر تو در میان کردم دست پنداشتمش که در...


In [211]:
poems_list = hafez_poems.poem.tolist()

In [224]:
hafez = ' '.join(poems_list)


In [228]:
hafez

'جز نقش تو در نظر نیامد ما را    جز کوی تو رهگذر نیامد ما را    خواب ارچه خوش آمد همه را در عهدت    حقا که به چشم در نیامد ما را بر گیر شراب طرب\u200cانگیز و بیا    پنهان ز رقیب سفله بستیز و بیا    مشنو سخن خصم که بنشین و مرو    بشنو ز من این نکته که برخیز و بیا گفتم که لبت، گفت لبم آب حیات    گفتم دهنت، گفت زهی حب نبات    گفتم سخن تو، گفت حافظ گفتا    شادی همه لطیفه گویان صلوات ماهی که قدش به سرو می\u200cماند راست    آیینه به دست و روی خود می\u200cآراست    دستارچه\u200cای پیشکشش کردم گفت    وصلم طلبی زهی خیالی که توراست من باکمر تو در میان کردم دست    پنداشتمش که در میان چیزی هست    پیداست از آن میان چو بربست کمر    تا من ز کمر چه طرف خواهم بربست تو بدری و خورشید تو را بنده شده\u200cست    تا بندهٔ تو شده\u200cست تابنده شده\u200cست    زان روی که از شعاع نور رخ تو    خورشید منیر و ماه تابنده شده\u200cست هر روز دلم به زیر باری دگر است    در دیدهٔ من ز هجر خاری دگر است    من جهد همی\u200cکنم قضا می\u200cگوید    بیرون ز کفایت تو کاری دگراست ماهم که رخش روشنی خور بگرفت    گرد خط او چشمهٔ کو

In [249]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 10000,oov_token ='<oov>')
tokenizer.fit_on_texts(poems_list)


In [250]:
num_words = len(tokenizer.word_index) + 1
num_words

9297

In [251]:
tokenizer.word_index

{'<oov>': 1,
 'که': 2,
 'و': 3,
 'به': 4,
 'از': 5,
 'در': 6,
 'تو': 7,
 'ز': 8,
 'آن': 9,
 'این': 10,
 'من': 11,
 'است': 12,
 'بر': 13,
 'را': 14,
 'دل': 15,
 'چه': 16,
 'ما': 17,
 'حافظ': 18,
 'ای': 19,
 'چو': 20,
 'سر': 21,
 'با': 22,
 'تا': 23,
 'هر': 24,
 'بود': 25,
 'گر': 26,
 'می': 27,
 'او': 28,
 'شد': 29,
 'نیست': 30,
 'چون': 31,
 'باد': 32,
 'جان': 33,
 'نه': 34,
 'گل': 35,
 'خوش': 36,
 'دست': 37,
 'عشق': 38,
 'چشم': 39,
 'کرد': 40,
 'خود': 41,
 'روی': 42,
 'اگر': 43,
 'یار': 44,
 'کن': 45,
 'غم': 46,
 'جهان': 47,
 'زلف': 48,
 'همه': 49,
 'جام': 50,
 'کار': 51,
 'گفت': 52,
 'دارد': 53,
 'خاک': 54,
 'کس': 55,
 'ره': 56,
 'دوست': 57,
 'لب': 58,
 'یک': 59,
 'آب': 60,
 'خون': 61,
 'کز': 62,
 'ساقی': 63,
 'آمد': 64,
 'دیده': 65,
 'دم': 66,
 'باده': 67,
 'یاد': 68,
 'بی': 69,
 'باشد': 70,
 'صبا': 71,
 'مرا': 72,
 'یا': 73,
 'خدا': 74,
 'کنم': 75,
 'راه': 76,
 'صد': 77,
 'کند': 78,
 'هم': 79,
 'سخن': 80,
 'شود': 81,
 'رخ': 82,
 'رفت': 83,
 'نظر': 84,
 'شب': 85,
 'بیا': 86,
 'روز': 8

In [252]:
tokenizer.texts_to_sequences('  ای بس که خراب باده و جام شوی ')

[[],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [3],
 [],
 [1],
 [1],
 [1],
 [],
 [1],
 [3],
 [1],
 []]

In [253]:
input_sequences = []
for poem in poem_list[:10]:
    token_list = tokenizer.texts_to_sequences(poem)
    print(token_list)

[[1], [1], [], [1], [1], [], [1], [1], [1], [], [1], [3], [1], [1], [1], [], [1], [3], [], [1], [1], [1], [1], [], [1], [1], [], [1], [1], [], [1], [3], [1], [], [], [], [], [1], [8], [1], [1], [], [1], [1], [1], [1], [], [3], [1], [1], [1], [1], [], [8], [], [1], [3], [], [1], [1], [1], [1], [], [1], [1], [1], [1], [], [], [], [], [1], [1], [1], [], [1], [1], [1], [], [1], [1], [3], [1], [1], [1], [], [1], [1], [], [1], [1], [1], [1], [1], [], [1], [1], [], [1], [1], [1], [1], [], [], [], [], [1], [1], [1], [], [1], [3], [1], [1], [1], [], [1], [1], [], [1], [1], [1], [1], [1], [], [1], [1], [], [1], [1], [1], [1], [], [1], [1], [], [1], [1], [1], [1], [], [], [], [], [1], [1], [], [1], [1], [], [1], [1], [], [1], [1], [1], [9295], [], [1], [1], [1], [], [1], [1], [1], [1], [], [1], [1], [1], [1], [], [1], [1], [1], [1], [], [], [], [], [1], [1], [], [1], [1], [], [1], [1], [], [1], [1], [1], [9295], [], [1], [1], [1], [], [1], [1], [1], [1], [], [1], [1], [1], [1], [], [1], [1], [1],

In [226]:
token_list

[24]

In [222]:
tokenizer.texts_to_sequences(poem_list[1])

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [ ]:
https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%201%20-%20Notebook.ipynb#scrollTo=soPGVheskaQP